In [28]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [29]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory+automation' # topic is "large language model"factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [31]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs: #stores all the problems in 1 string 
    user_prompt1 = f"""
    
    
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```    
   
    Take a deep breath before working on this problem. 
   
    You want to find pressing problems of your customers in the field of {query} which would be worth solving. You are business minded and want to come up with a problem which your team should focus on - as your customer would really appreciate a solution and pay for it. 
    
    For the problem you found, please only output the following (please write in total minimum 155 words): 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
        
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
     # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem Generation:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem Generation:

- "Title": Inefficient Production Line Optimization
- "Explanation": Many factories struggle with optimizing their production lines efficiently due to manual processes, leading to bottlenecks, delays, and increased operational costs. Lack of real-time data visibility and automation hinders productivity and scalability.
- "User group": Factory managers and production line supervisors.
- "Pain point": The user group faces challenges in meeting production targets, maintaining quality standards, and adapting quickly to market demands. Manual optimization processes result in decreased throughput, higher lead times, and lower overall equipment effectiveness, impacting profitability and competitiveness.

---

Problem Generation:

- "Title": Inefficient Inventory Management in Factories
- "Explanation": Many factories struggle with manual inventory management processes, leading to inefficiencies, inaccuracies, and delays in production. Without real-time visibility and auto

In [26]:
# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
   
    Read the following problems which you got as input: {problems_list} 
    
    Before you give me any output, please rate the problems according to the following criteria. Please be very critical in your ratings and only give a 3 rating when it is very very very good:
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 20 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")

Problem rating:

The problem is "Inefficient Production Line Integration."

- Size of user group: 2 (medium target group)
- Relevancy of pain point: 3 (very relevant to users)
- Specificity of the problem: 3 (specific and not generic problem)

Justification: This problem affects a significant number of production managers and operators in factories, directly impacting their efficiency and stress levels with non-integrated systems. Overall score: 8/9.

---

Problem rating:

**Problem:** Guest Check-in Automation Integration

- Size of user group: 2 (medium target group)
- Relevancy of pain point: 2 (medium relevant to users)
- Specificity of the problem: 3 (specific and not generic problem)

Justification: The problem addresses a medium-sized user group in the hospitality industry, with a pain point directly impacting hotel operations and guest experience. 

Overall score: 7/9

---

Problem rating:

**Title**: Inefficient Tech Transfer Processes

- Size of user group: 2 (medium target g

In [27]:
user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
    Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
        - The problem (please write 100-150 words)
        - Which user group is impacted by this problem the most (please write max 25 words)
        - What is the specific pain point for this user group resulting from the problem (please write 40-60 words)
        - One sentence that summarises the three selection criteria (Size of user group / Relevancy of pain point to the specific industry of {query} and the respective user group / How specific is the problem)


    """
    
completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print("Best Problem:\n")
print(completion.choices[0].message.content)
print("\n---\n")


Best Problem:

Based on the provided information, the problem with the highest overall rating is:

**Problem:** Downtime due to machinery breakdowns

- **The problem:** One pressing problem in factory+automation is the frequent downtime caused by unexpected machinery breakdowns, leading to production delays and loss of efficiency. This issue not only disrupts the production schedule but also incurs additional maintenance costs and affects overall productivity.

- **User group impacted the most:** Maintenance technicians and production managers are impacted the most by this problem.

- **Specific pain point for this user group:** Maintenance technicians face the challenge of identifying and resolving breakdowns promptly, while production managers struggle to meet deadlines and maintain optimal production output, impacting the bottom line of the company.

- **Summary of selection criteria:** This problem has a large target group, is very relevant to the factory+automation industry and th